## Importo librerias

In [1]:
import pandas as pd
import numpy as np
%pip install mysql-connector-python
import mysql.connector as conn

## Importacion de datos

In [2]:
actor = pd.read_csv(r'..\data\actor.csv',encoding='ISO-8859-1')
category = pd.read_csv(r'..\data\category.csv',encoding='ISO-8859-1')
film = pd.read_csv(r'..\data\film.csv',encoding='ISO-8859-1')
inventory = pd.read_csv(r'..\data\inventory.csv',encoding='ISO-8859-1')
language = pd.read_csv(r'..\data\language.csv',encoding='ISO-8859-1')
old_hdd = pd.read_csv(r'..\data\old_HDD.csv',encoding='ISO-8859-1')
rental = pd.read_csv(r'..\data\rental.csv',encoding='ISO-8859-1')

## Primera limpieza de los datos

In [3]:
def limpiar_dataframe(df):
    # Cambiar el formato de la columna last_update a tipo fecha (YYYY-MM-DD), quitando la hora.
    if 'last_update' in df.columns:
        df['last_update'] = pd.to_datetime(df['last_update'], format='%Y-%m-%d')
    
    # Cambiar el formato de las columnas a minuscula con la primera en mayuscula.
    # Solo aplicarlo para las columnas que sean contengan string.
    for column_name in df.select_dtypes(include=['object']).columns:
        df[column_name] = df[column_name].str.lower().str.capitalize()


    # Rellenar los valores nulos con 'NoInfo'.
    df.fillna('NoInfo', inplace=True)

In [4]:
# Limpio todas los dataframes.
limpiar_dataframe(actor)
limpiar_dataframe(category)
limpiar_dataframe(film)
limpiar_dataframe(inventory)
limpiar_dataframe(language)
limpiar_dataframe(old_hdd)
limpiar_dataframe(rental)

En el dataframe 'film' cambio los datos en la columna Language por las siglas. Segun la ISO 639-1 sera un string de dos letras.

In [5]:
# Como esta columna tiene como valor unico '1' lo cambio todo por 'EN'
film.loc[:, 'language_id'] = 'EN'

Junto first name y last name en una nueva columna que se llama full_name. Para que me sirva de nexo entre los dos data frame

In [6]:
actor['full_name'] = actor['last_name'] + ' ' + actor['first_name']
old_hdd['full_name'] = old_hdd['last_name'] + ' ' + old_hdd['first_name']


In [7]:
# Hago un left join de old_hdd con actor
old_hdd = old_hdd.merge(actor, on = 'full_name', how='left')

In [8]:
# Debido al merge, me ha creado todas las columnas que tenia actor dentro de old_hdd
old_hdd.columns

Index(['first_name_x', 'last_name_x', 'title', 'release_year', 'category_id',
       'full_name', 'actor_id', 'first_name_y', 'last_name_y', 'last_update'],
      dtype='object')

In [9]:
# Hago un left join de old_hdd con film.
old_hdd = old_hdd.merge(film, on= 'title', how='left')

In [10]:
# Debido al merge, me ha creado todas las columnas que tenia film dentro de old_hdd
old_hdd.columns

Index(['first_name_x', 'last_name_x', 'title', 'release_year_x', 'category_id',
       'full_name', 'actor_id', 'first_name_y', 'last_name_y', 'last_update_x',
       'film_id', 'description', 'release_year_y', 'language_id',
       'original_language_id', 'rental_duration', 'rental_rate', 'length',
       'replacement_cost', 'rating', 'special_features', 'last_update_y'],
      dtype='object')

In [11]:
# Hago un left join de old_hdd con film.
old_hdd = old_hdd.merge(category, on= 'category_id', how='left')

In [12]:
# Debido al merge, me ha creado todas las columnas que tenia category dentro de old_hdd
old_hdd.columns

Index(['first_name_x', 'last_name_x', 'title', 'release_year_x', 'category_id',
       'full_name', 'actor_id', 'first_name_y', 'last_name_y', 'last_update_x',
       'film_id', 'description', 'release_year_y', 'language_id',
       'original_language_id', 'rental_duration', 'rental_rate', 'length',
       'replacement_cost', 'rating', 'special_features', 'last_update_y',
       'name', 'last_update'],
      dtype='object')

In [13]:
# Hago un merge de film con el nuevo old_hdd que tiene ahora todas las columnas de film, actor y category.
# Esto me va a permitir añadir category a films.
# Esto va a generar duplicados en la nueva 'films'
film = film.merge(old_hdd, on= 'film_id', how='left')

C:\Users\carlo\AppData\Local\Temp\ipykernel_6028\3024531712.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'last_update_y'} in the result is deprecated and will raise a MergeError in a future version.
  film = film.merge(old_hdd, on= 'film_id', how='left')


In [14]:
film.columns

Index(['film_id', 'title_x', 'description_x', 'release_year', 'language_id_x',
       'original_language_id_x', 'rental_duration_x', 'rental_rate_x',
       'length_x', 'replacement_cost_x', 'rating_x', 'special_features_x',
       'last_update_x', 'first_name_x', 'last_name_x', 'title_y',
       'release_year_x', 'category_id', 'full_name', 'actor_id',
       'first_name_y', 'last_name_y', 'last_update_x', 'description_y',
       'release_year_y', 'language_id_y', 'original_language_id_y',
       'rental_duration_y', 'rental_rate_y', 'length_y', 'replacement_cost_y',
       'rating_y', 'special_features_y', 'last_update_y', 'name',
       'last_update_y'],
      dtype='object')

In [15]:
# Elimino todas las columnas duplicadas y original_language_id
film.drop(columns= ['original_language_id_x','title_y','release_year_x','full_name', 'actor_id','first_name_y',
                    'last_name_y', 'last_update_x', 'description_y','release_year_y', 'language_id_y', 'original_language_id_y',
                    'rental_duration_y', 'rental_rate_y', 'length_y', 'replacement_cost_y','rating_y', 'special_features_y',
                    'last_update_y', 'name','last_update_y','first_name_x', 'last_name_x'], inplace=True)

In [16]:
film.columns

Index(['film_id', 'title_x', 'description_x', 'release_year', 'language_id_x',
       'rental_duration_x', 'rental_rate_x', 'length_x', 'replacement_cost_x',
       'rating_x', 'special_features_x', 'category_id'],
      dtype='object')

In [17]:
# Aprovecho para cambiar nombres de columnas en film
film =film.rename(columns = {'title_x' : 'title', 'description_x':'description',  'language_id_x': 'language', 'rental_duration_x':'rental_duration',
                             'rental_rate_x':'rental_rate', 'length_x':'length', 'replacement_cost_x':'replacement_cost', 'rating_x':'rating',
                             'special_features_x':'special_features','category_id':'genre'})

En old_hdd solo me tengo que quedar con las dos Foreign Key de actor y film

In [18]:
#Comprobar que columnas tengo que eliminar para quedarme solo con las dos que necesito.
old_hdd.columns

Index(['first_name_x', 'last_name_x', 'title', 'release_year_x', 'category_id',
       'full_name', 'actor_id', 'first_name_y', 'last_name_y', 'last_update_x',
       'film_id', 'description', 'release_year_y', 'language_id',
       'original_language_id', 'rental_duration', 'rental_rate', 'length',
       'replacement_cost', 'rating', 'special_features', 'last_update_y',
       'name', 'last_update'],
      dtype='object')

In [19]:
# Elimino las columnas que no necesito.
old_hdd.drop(columns=['first_name_x', 'last_name_x', 'title', 'release_year_x', 'category_id',
       'full_name', 'first_name_y', 'last_name_y', 'last_update_x',
       'description', 'release_year_y', 'language_id',
       'original_language_id', 'rental_duration', 'rental_rate', 'length',
       'replacement_cost', 'rating', 'special_features', 'last_update_y',
       'name', 'last_update'] , inplace=True)

In [20]:
# Compruebo nuevo estado de old_hdd
old_hdd.columns

Index(['actor_id', 'film_id'], dtype='object')

Relacion de Inventory y Rental

In [21]:
# Le añado todas las columnas de inventory a rental
rental = rental.merge(inventory, on= 'inventory_id', how='left')

In [22]:
# Reviso nuevo estado de rental
rental.columns

Index(['rental_id', 'rental_date', 'inventory_id', 'customer_id',
       'return_date', 'staff_id', 'last_update_x', 'film_id', 'store_id',
       'last_update_y'],
      dtype='object')

In [23]:
# Ver que columnas puedo borrar y renombrar
rental.drop(columns = ['last_update_y','customer_id','staff_id','store_id'],inplace=True)

Como ya tengo todos los datos del stock metido en rental, el data frame inventory no me hace falta.

In [24]:
'''
## Cambio a tipo integer algunas columnas para que sean foreign keys
actors_film = actors_film.astype({'actors_actor_id':'int','film_film_id' : 'int' })
'''

"\n## Cambio a tipo integer algunas columnas para que sean foreign keys\nactors_film = actors_film.astype({'actors_actor_id':'int','film_film_id' : 'int' })\n"

In [25]:
# Tengo 386 duplicados en cada columna.
film.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1386 entries, 0 to 1385
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   film_id           1386 non-null   int64  
 1   title             1386 non-null   object 
 2   description       1386 non-null   object 
 3   release_year      1386 non-null   int64  
 4   language          1386 non-null   object 
 5   rental_duration   1386 non-null   int64  
 6   rental_rate       1386 non-null   float64
 7   length            1386 non-null   int64  
 8   replacement_cost  1386 non-null   float64
 9   rating            1386 non-null   object 
 10  special_features  1386 non-null   object 
 11  genre             1000 non-null   float64
dtypes: float64(3), int64(4), object(5)
memory usage: 140.8+ KB


In [26]:
film.duplicated().sum()

386

In [27]:
film.drop_duplicates( inplace=True)

In [28]:
# Compruebo duplicados de todos los data frame.
print(film.duplicated().sum())
print(actor.duplicated().sum())
print(old_hdd.duplicated().sum())
print(rental.duplicated().sum())

0
0
0
0


Creo una funcion para cambiar el id del genero por el nombre del genero.

In [29]:
film['genre'] = film['genre'].fillna(0).astype(int)

In [30]:
film.genre.unique()

array([ 6, 11,  0,  9, 15, 12,  4,  1,  2, 13,  8, 14,  7, 16, 10,  3,  5])

In [31]:
#Hago una funcion para que me cambie los numeros por los valores en texto.
def cambiar_genero(valor):
    if valor == 1:
        return 'Action'
    elif valor == 2:
        return 'Animation'
    elif valor == 3:
        return 'Children'
    elif valor == 4:
        return 'Classics'
    elif valor == 5:
        return 'Comedy'
    elif valor == 6:
        return 'Documentary'
    elif valor == 7:
        return 'Drama'
    elif valor == 8:
        return 'Family'
    elif valor == 9:
        return 'Foreign'
    elif valor == 10:
        return 'Games'
    elif valor == 11:
        return 'Horror'
    elif valor == 12:
        return 'Music'
    elif valor == 13:
        return 'New'
    elif valor == 14:
        return 'Sci-fi'
    elif valor == 15:
        return 'Sports'
    elif valor == 16:
        return 'Travel'
    else:
        return 'NoInfo'

In [32]:
# Aplico la funcion con .apply()
film['genre'] = film['genre'].apply(cambiar_genero)

Colocar, renombrar y eliminar las columnas de todos los Data Frame

In [33]:
film.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 1385
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   film_id           1000 non-null   int64  
 1   title             1000 non-null   object 
 2   description       1000 non-null   object 
 3   release_year      1000 non-null   int64  
 4   language          1000 non-null   object 
 5   rental_duration   1000 non-null   int64  
 6   rental_rate       1000 non-null   float64
 7   length            1000 non-null   int64  
 8   replacement_cost  1000 non-null   float64
 9   rating            1000 non-null   object 
 10  special_features  1000 non-null   object 
 11  genre             1000 non-null   object 
dtypes: float64(2), int64(4), object(6)
memory usage: 101.6+ KB


'actor'

In [34]:
actor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   actor_id     200 non-null    int64         
 1   first_name   200 non-null    object        
 2   last_name    200 non-null    object        
 3   last_update  200 non-null    datetime64[ns]
 4   full_name    200 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 7.9+ KB


In [35]:
old_hdd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   actor_id  1000 non-null   int64
 1   film_id   1000 non-null   int64
dtypes: int64(2)
memory usage: 23.4 KB


In [36]:
old_hdd.rename(columns={'actor_id': 'actor_actor_id', 'film_id': 'film_film_id'}, inplace=True)

In [37]:
# Cambio de nombre al data frame.
film_actor = old_hdd

Revision data frame rental.


In [38]:
# Solo estan metidos 212 titulos, de los 1000 segun el listado de peliculas de la tabla film
print(rental.film_id.max())
print(rental.film_id.min())
print(rental.film_id.count())

220.0
1.0
212


In [39]:
# Deberia tener la informacion de 4581 peliculas fisicas, pero solo tengo 1000 entradas de 212 peliculas.
print(rental.inventory_id.max())
print(rental.inventory_id.min())
print(rental.film_id.count())

4581
2
212


Conclusiones:

    Solo se han metido datos de 212 titulos, pero al haber mas de una pelicula se han añadido hasta tener 1000 registros.
    Necesitaremos el resto de titulos para completar la base de datos.

Acciones tomadas con la informacion de la que disponemos:

    Eliminar todas las filas que tengan nulos (o no tengan informacion) en la columna film_id.
    Si no tengo informacion sobre ellas, no las necesito.
    Mi nuevo rental_id sera el inventory_id que ira de 1 a 1000

In [40]:
rental.dropna(subset=['film_id'], inplace=True)

In [41]:
# Cambio a tipo integer estas dos columnas.
rental['film_id'] = rental['film_id'].astype(int)
rental['inventory_id'] = rental['inventory_id'].astype(int)

In [42]:
rental.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212 entries, 0 to 993
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   rental_id      212 non-null    int64         
 1   rental_date    212 non-null    object        
 2   inventory_id   212 non-null    int32         
 3   return_date    212 non-null    object        
 4   last_update_x  212 non-null    datetime64[ns]
 5   film_id        212 non-null    int32         
dtypes: datetime64[ns](1), int32(2), int64(1), object(2)
memory usage: 9.9+ KB


In [43]:
rental = rental.drop('rental_id', axis=1)

In [44]:
rental['rental_id'] = range(1, len(rental) + 1)
rental.set_index('rental_id', inplace=True)


In [45]:
rental.columns

Index(['rental_date', 'inventory_id', 'return_date', 'last_update_x',
       'film_id'],
      dtype='object')

In [46]:
rental.rename(columns={'film_id': 'film_film_id'}, inplace=True)

In [47]:
rental.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212 entries, 1 to 212
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   rental_date    212 non-null    object        
 1   inventory_id   212 non-null    int32         
 2   return_date    212 non-null    object        
 3   last_update_x  212 non-null    datetime64[ns]
 4   film_film_id   212 non-null    int32         
dtypes: datetime64[ns](1), int32(2), object(2)
memory usage: 8.3+ KB


## Exportar las tablas 'limpias'

In [49]:
actor.to_csv(r'..\export_csv\actor.csv', index=True)
film.to_csv(r'..\export_csv\film.csv', index=True)
inventory.to_csv(r'..\export_csv\inventory.csv', index=True)
film_actor.to_csv(r'..\export_csv\film_actor.csv', index=True)
rental.to_csv(r'..\export_csv\rental.csv', index=True)